In [7]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
from ultralytics import YOLO
import cv2
import os

model = YOLO("yolov8m.pt")  

input_dir = r'C:\Users\hussa\Downloads\beetroot detection YOLO\beetroot_images'
output_label_dir = 'labels'

os.makedirs(output_label_dir, exist_ok=True)

for img_file in os.listdir(input_dir):
    if img_file.endswith(('.jpg', '.png')):
        img_path = os.path.join(input_dir, img_file)
        results = model(img_path)
        boxes = results[0].boxes.xywhn  # normalized (x_center, y_center, width, height)

        label_file = os.path.join(output_label_dir, img_file.rsplit('.', 1)[0] + '.txt')
        with open(label_file, 'w') as f:
            for box in boxes:
                x_center, y_center, width, height = box[:4]
                f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")


image 1/1 C:\Users\hussa\Downloads\beetroot detection YOLO\beetroot_images\1.jpg: 640x480 (no detections), 532.1ms
Speed: 9.5ms preprocess, 532.1ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\hussa\Downloads\beetroot detection YOLO\beetroot_images\10.jpg: 640x480 1 vase, 344.9ms
Speed: 3.9ms preprocess, 344.9ms inference, 5.4ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\hussa\Downloads\beetroot detection YOLO\beetroot_images\11.jpg: 640x480 (no detections), 372.3ms
Speed: 4.6ms preprocess, 372.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\hussa\Downloads\beetroot detection YOLO\beetroot_images\13.jpg: 640x480 (no detections), 327.9ms
Speed: 3.6ms preprocess, 327.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\hussa\Downloads\beetroot detection YOLO\beetroot_images\14.jpg: 640x480 1 vase, 329.2ms
Speed: 2.8ms preprocess, 329.2ms inference

In [17]:
import os
import random
import shutil

# === Set up paths ===
base_dir = r"C:\Users\hussa\Downloads\beetroot detection YOLO"
images_dir = os.path.join(base_dir, "all_images")  # Your original images
labels_dir = os.path.join(base_dir, "all_labels")  # Your original YOLO label .txt files

# === Create train/val/test folders ===
splits = ["train", "val", "test"]
for split in splits:
    os.makedirs(os.path.join(base_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(base_dir, "labels", split), exist_ok=True)

# === Split ratios ===
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# === Get all image files ===
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
random.shuffle(image_files)

# === Compute split indices ===
total = len(image_files)
train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)

train_files = image_files[:train_end]
val_files = image_files[train_end:val_end]
test_files = image_files[val_end:]

# === Function to move files ===
def move_files(file_list, split):
    for file_name in file_list:
        # Move image
        src_img = os.path.join(images_dir, file_name)
        dst_img = os.path.join(base_dir, "images", split, file_name)
        shutil.copy2(src_img, dst_img)

        # Move label
        label_name = os.path.splitext(file_name)[0] + ".txt"
        src_label = os.path.join(labels_dir, label_name)
        dst_label = os.path.join(base_dir, "labels", split, label_name)
        if os.path.exists(src_label):
            shutil.copy2(src_label, dst_label)
        else:
            print(f"⚠️ Warning: No label found for {file_name}")

# === Move files into structured directories ===
move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

# === Create data.yaml ===
yaml_content = f"""path: {base_dir.replace(os.sep, "/")}

train: images/train
val: images/val
test: images/test

names:
  0: beetroot
"""

yaml_path = os.path.join(base_dir, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print("✅ Dataset split and `data.yaml` created successfully!")


✅ Dataset split and `data.yaml` created successfully!


In [15]:
pip install ultralytics

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")
model.train(data=r"C:\Users\hussa\Downloads\beetroot detection YOLO\data.yaml", epochs=10)



results = model.val(
    data=r"C:\Users\hussa\Downloads\BeetrootDetectorYOLO\data.yaml",
    split='test',
    save=True,         # Saves visualizations of predictions
    conf=0.25,         # Confidence threshold
    iou=0.5,           # IoU threshold for mAP calculation
    plots=True         # Generates confusion matrix, PR curve, etc.
)

# /*

# 📂 This Will Save:
# In the runs/val/ directory (e.g., runs/val/exp/), you’ll get:

# confusion_matrix.png – Visual confusion matrix

# F1_curve.png – F1 score vs confidence

# PR_curve.png – Precision-Recall curve

# labels.jpg – Dataset label distribution

# results.png – Combined training/validation metrics

# Annotated images with predictions

# results.csv – Raw metric values

# */


# Python Code to Display Evaluation Plots

import os
import cv2
import matplotlib.pyplot as plt

# Path to YOLOv8 validation results directory
val_dir = r"runs\val\exp"  # Update if it's exp2, exp3, etc.

# List of result images to display
image_files = [
    "confusion_matrix.png",
    "F1_curve.png",
    "PR_curve.png",
    "labels.jpg",
    "results.png"
]

# Function to display each image
def display_image(image_path, title):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load image: {image_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis("off")
    plt.title(title)
    plt.show()

# Loop through and display all result images
for img_file in image_files:
    full_path = os.path.join(val_dir, img_file)
    display_image(full_path, title=img_file.split(".")[0].replace("_", " ").title())